In [31]:
# 1. convert from many txt files to one csv file

import os
from tqdm import tqdm
import csv
import pickle

txts_path = "./save_stocks"

txt_paths = []
for root, dirs, files in os.walk(txts_path):
    for file in files:
        txt_paths.append(root + "/" + file)

# 2022 data
all = []
for path in tqdm(txt_paths):
    with open(path, "r", encoding='utf-8') as f:
        data = f.readlines()
        one = {}
        one['time'] = data[14].replace('\n', '')
        one['ticker_id'] = data[1].replace('\n', '')
        one['ticker_name'] = data[2].replace('\n', '')
        one['ticker_class'] = data[13].replace('\n', '')
        one['ticker_grade'] = [data[5].replace('\n', ''), data[6].replace('\n', '')]
        one['text_title'] = data[4].replace('\n', '')
        one['text_content'] = "".join(data[15:]).replace('\n', '').replace(' ', '').replace('\u3000\u3000', '')
        all.append(one)

with open("./files/eastmoney_stocks.pkl","wb") as f:
    pickle.dump(all, f)

100%|██████████| 4014/4014 [00:00<00:00, 5738.91it/s]


In [34]:
# 2. use baidu_api to analyze

import pandas as pd
from tqdm import tqdm
from utility.baidu_utils import BaiduNLU
import pickle

with open("./files/eastmoney_stocks.pkl","rb") as f:
    data = pickle.load(f)

print(len(data))

config = {}
config['APP_ID'] = '25576077'
config['API_KEY'] = 'iprdevAUKAE15tQkgEYpCyYQ'
config['SECRET_KEY'] = '64Zpk8fcpntYRsSyYfGmAoCMFvenGHi4'
nlu_tool = BaiduNLU(config)

all = []
for one in tqdm(data):

    title = one['text_title']
    content = one['text_content']

    # try:
    #     res_sentiment = nlu_tool.get_sentiment(content)
    # except Exception:
    #     res_sentiment = {}

    try:
        res_keyword = nlu_tool.get_keyword(title, content)
    except Exception:
        res_keyword = {}

    try:
        res_topic = nlu_tool.get_topic(title, content)
    except Exception:
        res_topic = {}

    # try:
    #     res_summary = nlu_tool.get_summary(title, content, maxSummaryLen=100)
    # except Exception:
    #     res_summary = {}

    one['text_keyword'] = res_keyword["items"] if "items" in res_keyword.keys() else None
    one['text_topic'] = res_topic["item"] if "item" in res_topic.keys() else None

    all.append(one)


with open("./files/eastmoney_stocks_nlu.pkl", "wb") as f:
    pickle.dump(all, f)

4014


100%|██████████| 4014/4014 [49:40<00:00,  1.35it/s]  


In [36]:

with open("./files/eastmoney_stocks_nlu.pkl", "rb") as f:
    all = pickle.load(f)

In [52]:
idx = 10
print(all[idx]['text_keyword'])
print(all[idx]['text_topic'])

[{'score': 0.941297, 'tag': '轴承'}, {'score': 0.861851, 'tag': '法兰轴承'}, {'score': 0.83386, 'tag': '股票'}, {'score': 0.799668, 'tag': '投资'}, {'score': 0.785835, 'tag': '供应商'}]
{'lv2_tag_list': [{'score': 0.784987, 'tag': '新能源'}, {'score': 0.781917, 'tag': '投资'}], 'lv1_tag_list': [{'score': 0.456114, 'tag': '财经'}]}
